#Pre-configuration

## Check Cuda memory (not necessary to execute)

In [1]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU


Gen RAM Free: 12.8 GB  | Proc size: 111.6 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


In [14]:
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 10.1 GB  | Proc size: 3.9 GB
GPU RAM Free: 14050MB | Used: 1029MB | Util   7% | Total 15079MB


In [ ]:
!kill -9 -1

## Mount Google Drive

In [1]:
# Set up colab instance
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Clone and pull github repository

In [2]:
# Make sure clone at root
%cd /content
!pip3 install pydicom
!git clone https://github.com/thomasp05/gif-705-projet

import os
os.chdir('gif-705-projet')

/content
fatal: destination path 'gif-705-projet' already exists and is not an empty directory.


In [39]:
!git checkout "features/test-unet" #develop
!git pull

M	models_parts.py
Branch 'features/test-unet' set up to track remote branch 'features/test-unet' from 'origin'.
Switched to a new branch 'features/test-unet'
Already up to date.


# Imports and load

In [40]:
import time

import torch

from dataset import *
from models import *
import models_parts

torch.manual_seed(111)

## HYPERPARAMETERS

In [5]:
N_EPOCH = 50
BATCH_SIZE = 4

## Load dataset

In [6]:
dataset = dcm_dataset('../drive/MyDrive/GIF-7005-Projet/gif-7005-projet/data')

train_set, test_set = train_test_split(dataset)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=BATCH_SIZE, num_workers=2)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=BATCH_SIZE, num_workers=2)

# Train

## CNN and Regressor

In [9]:
cnn = Vanilla_CNN(width=16).to("cuda:0")
regressor = Simple_regressor(16 * 8 * 8).to("cuda:0")

In [10]:
# If model exists, load it
if os.path.exists("../drive/MyDrive/GIF-7005-Projet/gif-7005-projet/cnn.pt"):
  cnn.load_state_dict(torch.load("../drive/MyDrive/GIF-7005-Projet/gif-7005-projet/cnn.pt"))
if os.path.exists("../drive/MyDrive/GIF-7005-Projet/gif-7005-projet/regressor.pt"):
  regressor.load_state_dict(torch.load("../drive/MyDrive/GIF-7005-Projet/gif-7005-projet/regressor.pt"))

In [11]:
params = list(cnn.parameters()) + list(regressor.parameters())
optim = torch.optim.Adam(params, lr=1e-4)

loss_fn = torch.nn.BCEWithLogitsLoss()

In [14]:
cnn.train()
regressor.train()

for epoch in range(N_EPOCH):
    timer = time.time()

    img_processed = 0

    for img, (target, bounding_box) in train_loader:

        optim.zero_grad()

        img, target = img.to("cuda:0"), target.to("cuda:0")

        pred1 = cnn(img)
        pred2 = regressor(pred1.flatten(1, -1)).squeeze()

        loss = loss_fn(pred2, target.to(torch.float))

        loss.backward()

        optim.step()

        img_processed += img.shape[0]

        # print("{:.2f} %, time : {:.2f}\r".format(100*img_processed/len(train_loader.dataset), time.time()-timer))

    print("Epoch : {}".format(epoch + 1))
    print("Time elapsed : {:.2f}".format(time.time() - timer))

KeyboardInterrupt: ignored

In [ ]:
# Save models
torch.save(cnn.state_dict(), "../drive/MyDrive/GIF-7005-Projet/gif-7005-projet/cnn.pt")
torch.save(regressor.state_dict(), "../drive/MyDrive/GIF-7005-Projet/gif-7005-projet/regressor.pt")

In [53]:
# Free memory
del optim, img, target, bounding_box
torch.cuda.empty_cache()

In [ ]:
# Accuracy

cnn.eval()
regressor.eval()

score = []

for img, (target, bounding_box) in test_loader:

    img, target = img.to("cuda:0"), target.to("cuda:0")

    pred1 = cnn(img)
    pred2 = regressor(pred1.flatten(1, -1)).squeeze()

    score_ = ((pred2 > 0.5) == (target == 1)).sum()

    score.append(score_)

score = torch.Tensor(score).sum() / len(test_loader.dataset)
print(score)

## U-Net

### Forget it

In [9]:

class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [10]:

import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

### Move the model to GPU if available

In [63]:
class ResNetUNet(nn.Module):
  def __init__(self, n_class):
    super().__init__()

    self.base_model = torchvision.models.resnet18(pretrained=True)
    self.base_layers = list(self.base_model.children())

    self.layer0 = nn.Sequential(*self.base_layers[:3]) # size=(N, 64, x.H/2, x.W/2)
    self.layer0_1x1 = conv_relu(64, 64, 1, 0)
    self.layer1 = nn.Sequential(*self.base_layers[3:5]) # size=(N, 64, x.H/4, x.W/4)
    self.layer1_1x1 = conv_relu(64, 64, 1, 0)
    self.layer2 = self.base_layers[5]  # size=(N, 128, x.H/8, x.W/8)
    self.layer2_1x1 = conv_relu(128, 128, 1, 0)
    self.layer3 = self.base_layers[6]  # size=(N, 256, x.H/16, x.W/16)
    self.layer3_1x1 = conv_relu(256, 256, 1, 0)
    self.layer4 = self.base_layers[7]  # size=(N, 512, x.H/32, x.W/32)
    self.layer4_1x1 = conv_relu(512, 512, 1, 0)

    self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    self.conv_up3 = conv_relu(256 + 512, 512, 3, 1)
    self.conv_up2 = conv_relu(128 + 512, 256, 3, 1)
    self.conv_up1 = conv_relu(64 + 256, 256, 3, 1)
    self.conv_up0 = conv_relu(64 + 256, 128, 3, 1)

    self.conv_original_size0 = conv_relu(1, 64, 3, 1)
    self.conv_original_size1 = conv_relu(64, 64, 3, 1)
    self.conv_original_size2 = conv_relu(64 + 128, 64, 3, 1)

    self.conv_last = nn.Conv2d(64, n_class, 1)

  def forward(self, input):
    x_original = self.conv_original_size0(input)
    x_original = self.conv_original_size1(x_original)

    layer0 = self.layer0(input)
    layer1 = self.layer1(layer0)
    layer2 = self.layer2(layer1)
    layer3 = self.layer3(layer2)
    layer4 = self.layer4(layer3)

    layer4 = self.layer4_1x1(layer4)
    x = self.upsample(layer4)
    layer3 = self.layer3_1x1(layer3)
    x = torch.cat([x, layer3], dim=1)
    x = self.conv_up3(x)

    x = self.upsample(x)
    layer2 = self.layer2_1x1(layer2)
    x = torch.cat([x, layer2], dim=1)
    x = self.conv_up2(x)

    x = self.upsample(x)
    layer1 = self.layer1_1x1(layer1)
    x = torch.cat([x, layer1], dim=1)
    x = self.conv_up1(x)

    x = self.upsample(x)
    layer0 = self.layer0_1x1(layer0)
    x = torch.cat([x, layer0], dim=1)
    x = self.conv_up0(x)

    x = self.upsample(x)
    x = torch.cat([x, x_original], dim=1)
    x = self.conv_original_size2(x)

    out = self.conv_last(x)

    return out


In [66]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device', device)

num_class = 1

#model = UNet(1,num_class).to(device)
model = ResNetUNet(num_class).to(device)

device cuda


## Train loop

Freeze backbone layers

In [67]:
for l in model.base_layers:
  for param in l.parameters():
    param.requires_grad = False

In [68]:
import torch.optim as optim
from torch.optim import lr_scheduler

optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=8, gamma=0.1)

In [47]:
dataloaders = {
  'train': train_loader,
  'val': test_loader
}

## new train

In [69]:
model.train()

optim = optimizer_ft
loss_fn = torch.nn.BCEWithLogitsLoss()

for epoch in range(N_EPOCH):
    timer = time.time()

    img_processed = 0

    for img, (target, bounding_box) in train_loader:

        optim.zero_grad()

        img, target = img.to("cuda:0"), target.to("cuda:0")

        pred1 = model(img)

        loss = loss_fn(pred1, target.to(torch.float))

        loss.backward()

        optim.step()

        img_processed += img.shape[0]

        # print("{:.2f} %, time : {:.2f}\r".format(100*img_processed/len(train_loader.dataset), time.time()-timer))

    print("Epoch : {}".format(epoch + 1))
    print("Time elapsed : {:.2f}".format(time.time() - timer))

RuntimeError: ignored

## old train

In [13]:
from collections import defaultdict

def train_model(model, optimizer, scheduler, num_epochs = 25, checkpoint_path = "checkpoint.pt"):
    best_loss = 1e10

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0

            for inputs, (labels, _) in dataloaders[phase]:
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = calc_loss(outputs, labels, metrics)
                    del outputs, labels
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        del loss

                # statistics
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples

            if phase == 'train':
              scheduler.step()
              for param_group in optimizer.param_groups:
                  print("LR", param_group['lr'])

            # save the model weights
            if phase == 'val' and epoch_loss < best_loss:
                print(f"saving best model to {checkpoint_path}")
                best_loss = epoch_loss
                torch.save(model.state_dict(), checkpoint_path)

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(torch.load(checkpoint_path))
    return model

In [14]:
checkpoint_path = "unet.pt"
model = train_model(model, optimizer_ft, exp_lr_scheduler, 10, checkpoint_path)

Epoch 0/9
----------


RuntimeError: ignored

In [1]:
torch.cuda.empty_cache()

NameError: ignored